# Agenda

### 0. Przygotowanie
### 1. Wprowadzenie do Snowflake
### 2. Przegląd funkcji 
### 3. Snowflake cortex
### 4. Copilot
### 5 Cortex Search
### 6. Cortex Analyst
### 7. Cortex Agents

# Zanim wyruszymy

## Kod SQL
Przed dalszą pracą w obrębie notebooka wejdź w wokrsheets i puść poniższy **kod SQL**.
-- opis co to i dlaczego to maja zrobic i jak -> pokazanie a potem przejscie do czesci teoretycznej

## Paczki

Kod pythonowy będzie wymagał kilka paczek do zainstalowania, aby je dodać kliknij na **Packages**, następnie wyszukaj potrzebne paczki i kliknij na nie by je dodać.

Dodaj:
 - streamlit
 - matplotlib

---

![packages](https://i.ibb.co/Y477Sfh2/packages.png)


In [ ]:
-- opis co to i dlaczego to maja zrobic i jak -> pokazanie a potem przejscie do czesci teoretycznej

USE ROLE sysadmin;

  /*--
  • database, schema and warehouse creation
  --*/

  -- create tb_voc database
  CREATE OR REPLACE DATABASE tb_voc;

  -- create raw_pos schema
  CREATE OR REPLACE SCHEMA tb_voc.raw_pos;

  -- create raw_customer schema
  CREATE OR REPLACE SCHEMA tb_voc.raw_support;

  -- create harmonized schema
  CREATE OR REPLACE SCHEMA tb_voc.harmonized;

  -- create analytics schema
  CREATE OR REPLACE SCHEMA tb_voc.analytics;

  -- create tasty_ds_wh warehouse
  CREATE OR REPLACE WAREHOUSE tasty_ds_wh
      WAREHOUSE_SIZE = 'large'
      WAREHOUSE_TYPE = 'standard'
      AUTO_SUSPEND = 60
      AUTO_RESUME = TRUE
      INITIALLY_SUSPENDED = TRUE
  COMMENT = 'data science warehouse for tasty bytes';


  USE WAREHOUSE tasty_ds_wh;

  /*--
  • file format and stage creation
  --*/

  CREATE OR REPLACE FILE FORMAT tb_voc.public.csv_ff 
  type = 'csv';

  CREATE OR REPLACE STAGE tb_voc.public.s3load
  COMMENT = 'Quickstarts S3 Stage Connection'
  url = 's3://sfquickstarts/tastybytes-voc/'
  file_format = tb_voc.public.csv_ff;

  /*--
  raw zone table build 
  --*/

  -- menu table build
  CREATE OR REPLACE TABLE tb_voc.raw_pos.menu
  (
      menu_id NUMBER(19,0),
      menu_type_id NUMBER(38,0),
      menu_type VARCHAR(16777216),
      truck_brand_name VARCHAR(16777216),
      menu_item_id NUMBER(38,0),
      menu_item_name VARCHAR(16777216),
      item_category VARCHAR(16777216),
      item_subcategory VARCHAR(16777216),
      cost_of_goods_usd NUMBER(38,4),
      sale_price_usd NUMBER(38,4),
      menu_item_health_metrics_obj VARIANT
  );

  -- truck table build 
  CREATE OR REPLACE TABLE tb_voc.raw_pos.truck
  (
      truck_id NUMBER(38,0),
      menu_type_id NUMBER(38,0),
      primary_city VARCHAR(16777216),
      region VARCHAR(16777216),
      iso_region VARCHAR(16777216),
      country VARCHAR(16777216),
      iso_country_code VARCHAR(16777216),
      franchise_flag NUMBER(38,0),
      year NUMBER(38,0),
      make VARCHAR(16777216),
      model VARCHAR(16777216),
      ev_flag NUMBER(38,0),
      franchise_id NUMBER(38,0),
      truck_opening_date DATE
  );

  -- order_header table build
  CREATE OR REPLACE TABLE tb_voc.raw_pos.order_header
  (
      order_id NUMBER(38,0),
      truck_id NUMBER(38,0),
      location_id FLOAT,
      customer_id NUMBER(38,0),
      discount_id VARCHAR(16777216),
      shift_id NUMBER(38,0),
      shift_start_time TIME(9),
      shift_end_time TIME(9),
      order_channel VARCHAR(16777216),
      order_ts TIMESTAMP_NTZ(9),
      served_ts VARCHAR(16777216),
      order_currency VARCHAR(3),
      order_amount NUMBER(38,4),
      order_tax_amount VARCHAR(16777216),
      order_discount_amount VARCHAR(16777216),
      order_total NUMBER(38,4)
  );

  -- truck_reviews table build
  CREATE OR REPLACE TABLE tb_voc.raw_support.truck_reviews
  (
      order_id NUMBER(38,0),
      language VARCHAR(16777216),
      source VARCHAR(16777216),
      review VARCHAR(16777216),
      review_id NUMBER(18,0)
  );

  /*--
  • harmonized view creation
  --*/

  -- truck_reviews_v view
  CREATE OR REPLACE VIEW tb_voc.harmonized.truck_reviews_v
      AS
  SELECT DISTINCT
      r.review_id,
      r.order_id,
      oh.truck_id,
      r.language,
      source,
      r.review,
      t.primary_city,
      oh.customer_id,
      TO_DATE(oh.order_ts) AS date,
      m.truck_brand_name
  FROM tb_voc.raw_support.truck_reviews r
  JOIN tb_voc.raw_pos.order_header oh
      ON oh.order_id = r.order_id
  JOIN tb_voc.raw_pos.truck t
      ON t.truck_id = oh.truck_id
  JOIN tb_voc.raw_pos.menu m
      ON m.menu_type_id = t.menu_type_id;

  /*--
  • analytics view creation
  --*/

  -- truck_reviews_v view
  CREATE OR REPLACE VIEW tb_voc.analytics.truck_reviews_v
      AS
  SELECT * FROM harmonized.truck_reviews_v;


  /*--
  raw zone table load 
  --*/


  -- menu table load
  COPY INTO tb_voc.raw_pos.menu
  FROM @tb_voc.public.s3load/raw_pos/menu/;

  -- truck table load
  COPY INTO tb_voc.raw_pos.truck
  FROM @tb_voc.public.s3load/raw_pos/truck/;

  -- order_header table load
  COPY INTO tb_voc.raw_pos.order_header
  FROM @tb_voc.public.s3load/raw_pos/order_header/;

  -- truck_reviews table load
  COPY INTO tb_voc.raw_support.truck_reviews
  FROM @tb_voc.public.s3load/raw_support/truck_reviews/;


  -- scale wh to medium
  ALTER WAREHOUSE tasty_ds_wh SET WAREHOUSE_SIZE = 'Medium';


  CREATE OR REPLACE TABLE CONCATENATED_REVIEWS AS
  WITH RANKED_REVIEWS AS (
      SELECT 
          TRUCK_BRAND_NAME,
          REVIEW,
          ROW_NUMBER() OVER (PARTITION BY TRUCK_BRAND_NAME ORDER BY REVIEW) AS ROW_NUM
      FROM TRUCK_REVIEWS_V
  ),
  FILTERED_REVIEWS AS (
      SELECT *
      FROM RANKED_REVIEWS
      WHERE ROW_NUM <= 20
  ),
  AGGREGATED_REVIEWS AS (
      SELECT 
          TRUCK_BRAND_NAME,
          ARRAY_AGG(REVIEW) AS ALL_REVIEWS
      FROM FILTERED_REVIEWS
      GROUP BY TRUCK_BRAND_NAME
  ),
  CONCATENATED_REVIEWS AS (
      SELECT 
          TRUCK_BRAND_NAME,
          ARRAY_TO_STRING(ALL_REVIEWS, ' ') AS ALL_REVIEWS_TEXT
      FROM AGGREGATED_REVIEWS
  )

  SELECT * FROM CONCATENATED_REVIEWS;

-- setup completion note
SELECT 'Setup is complete' AS note;
  SELECT * FROM CONCATENATED_REVIEWS;
select * , SNOWFLAKE.CORTEX.SENTIMENT(review) as sentiment_of_review from TRUCK_REVIEWS_V limit 10;

In [ ]:
USE ROLE sysadmin;

-- assign Query Tag to Session 
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"tb_zts","version":{"major":1, "minor":1},"attributes":{"is_quickstart":1, "source":"sql", "vignette": "intro"}}';

/*--
 • database, schema and warehouse creation
--*/

-- create tb_101 database
CREATE OR REPLACE DATABASE tb_101;

-- create raw_pos schema
CREATE OR REPLACE SCHEMA tb_101.raw_pos;

-- create raw_customer schema
CREATE OR REPLACE SCHEMA tb_101.raw_customer;

-- create harmonized schema
CREATE OR REPLACE SCHEMA tb_101.harmonized;

-- create analytics schema
CREATE OR REPLACE SCHEMA tb_101.analytics;

-- create warehouses
CREATE OR REPLACE WAREHOUSE tb_de_wh
    WAREHOUSE_SIZE = 'large' -- Large for initial data load - scaled down to XSmall at end of this scripts
    WAREHOUSE_TYPE = 'standard'
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
COMMENT = 'data engineering warehouse for tasty bytes';

CREATE OR REPLACE WAREHOUSE tb_dev_wh
    WAREHOUSE_SIZE = 'xsmall'
    WAREHOUSE_TYPE = 'standard'
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
COMMENT = 'developer warehouse for tasty bytes';

-- create roles
USE ROLE securityadmin;

-- functional roles
CREATE ROLE IF NOT EXISTS tb_admin
    COMMENT = 'admin for tasty bytes';
    
CREATE ROLE IF NOT EXISTS tb_data_engineer
    COMMENT = 'data engineer for tasty bytes';
    
CREATE ROLE IF NOT EXISTS tb_dev
    COMMENT = 'developer for tasty bytes';
    
-- role hierarchy
GRANT ROLE tb_admin TO ROLE sysadmin;
GRANT ROLE tb_data_engineer TO ROLE tb_admin;
GRANT ROLE tb_dev TO ROLE tb_data_engineer;

-- privilege grants
USE ROLE accountadmin;

GRANT IMPORTED PRIVILEGES ON DATABASE snowflake TO ROLE tb_data_engineer;

GRANT CREATE WAREHOUSE ON ACCOUNT TO ROLE tb_admin;

USE ROLE securityadmin;

GRANT USAGE ON DATABASE tb_101 TO ROLE tb_admin;
GRANT USAGE ON DATABASE tb_101 TO ROLE tb_data_engineer;
GRANT USAGE ON DATABASE tb_101 TO ROLE tb_dev;

GRANT USAGE ON ALL SCHEMAS IN DATABASE tb_101 TO ROLE tb_admin;
GRANT USAGE ON ALL SCHEMAS IN DATABASE tb_101 TO ROLE tb_data_engineer;
GRANT USAGE ON ALL SCHEMAS IN DATABASE tb_101 TO ROLE tb_dev;

GRANT ALL ON SCHEMA tb_101.raw_pos TO ROLE tb_admin;
GRANT ALL ON SCHEMA tb_101.raw_pos TO ROLE tb_data_engineer;
GRANT ALL ON SCHEMA tb_101.raw_pos TO ROLE tb_dev;

GRANT ALL ON SCHEMA tb_101.harmonized TO ROLE tb_admin;
GRANT ALL ON SCHEMA tb_101.harmonized TO ROLE tb_data_engineer;
GRANT ALL ON SCHEMA tb_101.harmonized TO ROLE tb_dev;

GRANT ALL ON SCHEMA tb_101.analytics TO ROLE tb_admin;
GRANT ALL ON SCHEMA tb_101.analytics TO ROLE tb_data_engineer;
GRANT ALL ON SCHEMA tb_101.analytics TO ROLE tb_dev;

-- warehouse grants
GRANT OWNERSHIP ON WAREHOUSE tb_de_wh TO ROLE tb_admin COPY CURRENT GRANTS;
GRANT ALL ON WAREHOUSE tb_de_wh TO ROLE tb_admin;
GRANT ALL ON WAREHOUSE tb_de_wh TO ROLE tb_data_engineer;

GRANT ALL ON WAREHOUSE tb_dev_wh TO ROLE tb_admin;
GRANT ALL ON WAREHOUSE tb_dev_wh TO ROLE tb_data_engineer;
GRANT ALL ON WAREHOUSE tb_dev_wh TO ROLE tb_dev;

-- future grants
GRANT ALL ON FUTURE TABLES IN SCHEMA tb_101.raw_pos TO ROLE tb_admin;
GRANT ALL ON FUTURE TABLES IN SCHEMA tb_101.raw_pos TO ROLE tb_data_engineer;
GRANT ALL ON FUTURE TABLES IN SCHEMA tb_101.raw_pos TO ROLE tb_dev;

GRANT ALL ON FUTURE TABLES IN SCHEMA tb_101.raw_customer TO ROLE tb_admin;
GRANT ALL ON FUTURE TABLES IN SCHEMA tb_101.raw_customer TO ROLE tb_data_engineer;
GRANT ALL ON FUTURE TABLES IN SCHEMA tb_101.raw_customer TO ROLE tb_dev;

GRANT ALL ON FUTURE VIEWS IN SCHEMA tb_101.harmonized TO ROLE tb_admin;
GRANT ALL ON FUTURE VIEWS IN SCHEMA tb_101.harmonized TO ROLE tb_data_engineer;
GRANT ALL ON FUTURE VIEWS IN SCHEMA tb_101.harmonized TO ROLE tb_dev;

GRANT ALL ON FUTURE VIEWS IN SCHEMA tb_101.analytics TO ROLE tb_admin;
GRANT ALL ON FUTURE VIEWS IN SCHEMA tb_101.analytics TO ROLE tb_data_engineer;
GRANT ALL ON FUTURE VIEWS IN SCHEMA tb_101.analytics TO ROLE tb_dev;

-- Apply Masking Policy Grants
USE ROLE accountadmin;
GRANT APPLY MASKING POLICY ON ACCOUNT TO ROLE tb_admin;
GRANT APPLY MASKING POLICY ON ACCOUNT TO ROLE tb_data_engineer;
  
-- raw_pos table build
USE ROLE sysadmin;
USE WAREHOUSE tb_de_wh;

/*--
 • file format and stage creation
--*/

CREATE OR REPLACE FILE FORMAT tb_101.public.csv_ff 
type = 'csv';

CREATE OR REPLACE STAGE tb_101.public.s3load
COMMENT = 'Quickstarts S3 Stage Connection'
url = 's3://sfquickstarts/frostbyte_tastybytes/'
file_format = tb_101.public.csv_ff;

/*--
 raw zone table build 
--*/

-- country table build
CREATE OR REPLACE TABLE tb_101.raw_pos.country
(
    country_id NUMBER(18,0),
    country VARCHAR(16777216),
    iso_currency VARCHAR(3),
    iso_country VARCHAR(2),
    city_id NUMBER(19,0),
    city VARCHAR(16777216),
    city_population VARCHAR(16777216)
);

-- franchise table build
CREATE OR REPLACE TABLE tb_101.raw_pos.franchise 
(
    franchise_id NUMBER(38,0),
    first_name VARCHAR(16777216),
    last_name VARCHAR(16777216),
    city VARCHAR(16777216),
    country VARCHAR(16777216),
    e_mail VARCHAR(16777216),
    phone_number VARCHAR(16777216) 
);

-- location table build
CREATE OR REPLACE TABLE tb_101.raw_pos.location
(
    location_id NUMBER(19,0),
    placekey VARCHAR(16777216),
    location VARCHAR(16777216),
    city VARCHAR(16777216),
    region VARCHAR(16777216),
    iso_country_code VARCHAR(16777216),
    country VARCHAR(16777216)
);

-- menu table build
CREATE OR REPLACE TABLE tb_101.raw_pos.menu
(
    menu_id NUMBER(19,0),
    menu_type_id NUMBER(38,0),
    menu_type VARCHAR(16777216),
    truck_brand_name VARCHAR(16777216),
    menu_item_id NUMBER(38,0),
    menu_item_name VARCHAR(16777216),
    item_category VARCHAR(16777216),
    item_subcategory VARCHAR(16777216),
    cost_of_goods_usd NUMBER(38,4),
    sale_price_usd NUMBER(38,4),
    menu_item_health_metrics_obj VARIANT
);

-- truck table build 
CREATE OR REPLACE TABLE tb_101.raw_pos.truck
(
    truck_id NUMBER(38,0),
    menu_type_id NUMBER(38,0),
    primary_city VARCHAR(16777216),
    region VARCHAR(16777216),
    iso_region VARCHAR(16777216),
    country VARCHAR(16777216),
    iso_country_code VARCHAR(16777216),
    franchise_flag NUMBER(38,0),
    year NUMBER(38,0),
    make VARCHAR(16777216),
    model VARCHAR(16777216),
    ev_flag NUMBER(38,0),
    franchise_id NUMBER(38,0),
    truck_opening_date DATE
);

-- order_header table build
CREATE OR REPLACE TABLE tb_101.raw_pos.order_header
(
    order_id NUMBER(38,0),
    truck_id NUMBER(38,0),
    location_id FLOAT,
    customer_id NUMBER(38,0),
    discount_id VARCHAR(16777216),
    shift_id NUMBER(38,0),
    shift_start_time TIME(9),
    shift_end_time TIME(9),
    order_channel VARCHAR(16777216),
    order_ts TIMESTAMP_NTZ(9),
    served_ts VARCHAR(16777216),
    order_currency VARCHAR(3),
    order_amount NUMBER(38,4),
    order_tax_amount VARCHAR(16777216),
    order_discount_amount VARCHAR(16777216),
    order_total NUMBER(38,4)
);

-- order_detail table build
CREATE OR REPLACE TABLE tb_101.raw_pos.order_detail 
(
    order_detail_id NUMBER(38,0),
    order_id NUMBER(38,0),
    menu_item_id NUMBER(38,0),
    discount_id VARCHAR(16777216),
    line_number NUMBER(38,0),
    quantity NUMBER(5,0),
    unit_price NUMBER(38,4),
    price NUMBER(38,4),
    order_item_discount_amount VARCHAR(16777216)
);

-- customer loyalty table build
CREATE OR REPLACE TABLE tb_101.raw_customer.customer_loyalty
(
    customer_id NUMBER(38,0),
    first_name VARCHAR(16777216),
    last_name VARCHAR(16777216),
    city VARCHAR(16777216),
    country VARCHAR(16777216),
    postal_code VARCHAR(16777216),
    preferred_language VARCHAR(16777216),
    gender VARCHAR(16777216),
    favourite_brand VARCHAR(16777216),
    marital_status VARCHAR(16777216),
    children_count VARCHAR(16777216),
    sign_up_date DATE,
    birthday_date DATE,
    e_mail VARCHAR(16777216),
    phone_number VARCHAR(16777216)
);

/*--
 • harmonized view creation
--*/

-- orders_v view
CREATE OR REPLACE VIEW tb_101.harmonized.orders_v
    AS
SELECT 
    oh.order_id,
    oh.truck_id,
    oh.order_ts,
    od.order_detail_id,
    od.line_number,
    m.truck_brand_name,
    m.menu_type,
    t.primary_city,
    t.region,
    t.country,
    t.franchise_flag,
    t.franchise_id,
    f.first_name AS franchisee_first_name,
    f.last_name AS franchisee_last_name,
    l.location_id,
    cl.customer_id,
    cl.first_name,
    cl.last_name,
    cl.e_mail,
    cl.phone_number,
    cl.children_count,
    cl.gender,
    cl.marital_status,
    od.menu_item_id,
    m.menu_item_name,
    od.quantity,
    od.unit_price,
    od.price,
    oh.order_amount,
    oh.order_tax_amount,
    oh.order_discount_amount,
    oh.order_total
FROM tb_101.raw_pos.order_detail od
JOIN tb_101.raw_pos.order_header oh
    ON od.order_id = oh.order_id
JOIN tb_101.raw_pos.truck t
    ON oh.truck_id = t.truck_id
JOIN tb_101.raw_pos.menu m
    ON od.menu_item_id = m.menu_item_id
JOIN tb_101.raw_pos.franchise f
    ON t.franchise_id = f.franchise_id
JOIN tb_101.raw_pos.location l
    ON oh.location_id = l.location_id
LEFT JOIN tb_101.raw_customer.customer_loyalty cl
    ON oh.customer_id = cl.customer_id;

-- loyalty_metrics_v view
CREATE OR REPLACE VIEW tb_101.harmonized.customer_loyalty_metrics_v
    AS
SELECT 
    cl.customer_id,
    cl.city,
    cl.country,
    cl.first_name,
    cl.last_name,
    cl.phone_number,
    cl.e_mail,
    SUM(oh.order_total) AS total_sales,
    ARRAY_AGG(DISTINCT oh.location_id) AS visited_location_ids_array
FROM tb_101.raw_customer.customer_loyalty cl
JOIN tb_101.raw_pos.order_header oh
ON cl.customer_id = oh.customer_id
GROUP BY cl.customer_id, cl.city, cl.country, cl.first_name,
cl.last_name, cl.phone_number, cl.e_mail;

/*--
 • analytics view creation
--*/

-- orders_v view
CREATE OR REPLACE VIEW tb_101.analytics.orders_v
COMMENT = 'Tasty Bytes Order Detail View'
    AS
SELECT DATE(o.order_ts) AS date, * FROM tb_101.harmonized.orders_v o;

-- customer_loyalty_metrics_v view
CREATE OR REPLACE VIEW tb_101.analytics.customer_loyalty_metrics_v
COMMENT = 'Tasty Bytes Customer Loyalty Member Metrics View'
    AS
SELECT * FROM tb_101.harmonized.customer_loyalty_metrics_v;

/*--
 raw zone table load 
--*/

-- country table load
COPY INTO tb_101.raw_pos.country
FROM @tb_101.public.s3load/raw_pos/country/;

-- franchise table load
COPY INTO tb_101.raw_pos.franchise
FROM @tb_101.public.s3load/raw_pos/franchise/;

-- location table load
COPY INTO tb_101.raw_pos.location
FROM @tb_101.public.s3load/raw_pos/location/;

-- menu table load
COPY INTO tb_101.raw_pos.menu
FROM @tb_101.public.s3load/raw_pos/menu/;

-- truck table load
COPY INTO tb_101.raw_pos.truck
FROM @tb_101.public.s3load/raw_pos/truck/;

-- customer_loyalty table load
COPY INTO tb_101.raw_customer.customer_loyalty
FROM @tb_101.public.s3load/raw_customer/customer_loyalty/;

-- order_header table load
COPY INTO tb_101.raw_pos.order_header
FROM @tb_101.public.s3load/raw_pos/order_header/;

-- order_detail table load
COPY INTO tb_101.raw_pos.order_detail
FROM @tb_101.public.s3load/raw_pos/order_detail/;

ALTER WAREHOUSE tb_de_wh SET WAREHOUSE_SIZE = 'XSmall';
create stage TB_101.ANALYTICS.MODELS;
-- setup completion note
SELECT 'tb_101 setup is now complete' AS note;

# Snowflake: Platforma danych w chmurze

## Czym jest Snowflake?

**Snowflake** to chmurowa platforma danych, która umożliwia jednoczesne przechowywanie, przetwarzanie i analizę danych w jednym środowisku. Dzięki architekturze opartej na rozdzieleniu zasobów storage i compute, Snowflake pozwala na elastyczne i efektywne skalowanie zgodnie z bieżącymi potrzebami. Snowflake jest jednym z liderów w obszarze hurtowni danych i analityki w chmurze, często wymienianym w ścisłej czołówce obok takich rozwiązań jak Amazon Redshift, Google BigQuery czy Azure Synapse.

![architecture](https://docs.snowflake.com/en/_images/architecture-overview.png)

---

## Zalety Snowflake

Snowflake to chmurowa platforma danych oferująca rozwiązania do przechowywania, przetwarzania i analizy dużych zbiorów danych. Do jej głównych zalet należą:

1. **Elastyczność i skalowalność**:
   - Umożliwia szybkie skalowanie zasobów w górę lub w dół w zależności od aktualnych potrzeb, co jest kluczowe w przypadku zmiennego zapotrzebowania na przetwarzanie danych.

2. **Wydajność**:
   - Oddzielenie przetwarzania od przechowywania danych zapewnia efektywność i zmniejsza ryzyko przestojów.

3. **Integracja z narzędziami analitycznymi**:
   - Snowflake integruje się z popularnymi narzędziami do analizy danych, takimi jak Tableau, Power BI czy Python, umożliwiając łatwe przesyłanie i przetwarzanie danych z różnych źródeł.

---

## O czym warto pamiętać?

Snowflake ma również pewne ograniczenia:

1. **Koszty**:
   - Przy intensywnym korzystaniu z zasobów obliczeniowych, koszty mogą szybko wzrosnąć, mimo elastycznego modelu płatności. 

2. **Krzywa uczenia się**:
   - Niektóre zaawansowane funkcje mogą wymagać czasu na opanowanie, pomimo ogólnej łatwości użycia. 

---

**Gotowy, aby zacząć?** Snowflake czeka, aby pomóc Ci w transformacji danych! 🚀

# Co potrafi Snowflake?

Poniżej znajdziesz dwa przykłady do czego jest zdolny Snowflake i jak ułatwia pracę.

---

## Przykład: Prognoza śniegu na weekend

Ten przykład pokazuje, jak stworzyć bazę danych, schemę i tabelę za pomocą SQL w Snowflake Notebooks.

In [ ]:
use role accountadmin;
-- Tworzenie bazy danych
CREATE or replace DATABASE university_db;

-- Użycie bazy danych
USE DATABASE university_db;

-- Tworzenie schematu
CREATE or replace SCHEMA university_schema;

-- Użycie schematu
USE SCHEMA university_schema;

-- Tworzenie tabeli wykładów
CREATE or replace TABLE lectures (
    lecture_name STRING NOT NULL,
    lecturer STRING NOT NULL,
    lecture_date DATE NOT NULL,
    start_time TIME NOT NULL,
    end_time TIME NOT NULL,
    room_number STRING,
    capacity INT
);


In [ ]:
INSERT INTO lectures (lecture_name, lecturer, lecture_date, start_time, end_time, room_number, capacity) 
VALUES 
    ('Operating Systems', 'Dr. White', '2025-03-22', '16:30:00', '16:30:00', 'B202', 51),
    ('Big Data Analytics', 'Dr. White', '2025-03-16', '16:00:00', '16:30:00', 'B202', 128),
    ('Big Data Analytics', 'Dr. Brown', '2025-03-12', '14:00:00', '10:00:00', 'A101', 64),
    ('Blockchain Technology', 'Dr. Brown', '2025-03-24', '18:00:00', '12:30:00', 'A101', 172),
    ('Machine Learning', 'Dr. White', '2025-03-11', '13:00:00', '10:00:00', 'C303', 111),
    ('Cyber Security', 'Dr. Smith', '2025-03-14', '09:30:00', '11:00:00', 'D404', 95),
    ('Database Systems', 'Dr. Johnson', '2025-03-19', '10:00:00', '11:30:00', 'A101', 85),
    ('Cloud Computing', 'Dr. Brown', '2025-03-20', '12:00:00', '13:30:00', 'B202', 120),
    ('Data Structures', 'Prof. Johnson', '2025-03-21', '14:00:00', '15:30:00', 'C303', 100),
    ('Algorithms', 'Dr. Green', '2025-03-22', '16:00:00', '17:30:00', 'D404', 130),
    ('Software Engineering', 'Dr. Smith', '2025-03-23', '09:00:00', '10:30:00', 'A101', 140),
    ('Computer Networks', 'Prof. Johnson', '2025-03-24', '11:00:00', '12:30:00', 'B202', 110),
    ('Artificial Intelligence', 'Dr. White', '2025-03-25', '13:00:00', '14:30:00', 'C303', 90),
    ('Parallel Computing', 'Dr. Brown', '2025-03-26', '15:00:00', '16:30:00', 'D404', 105),
    ('Human-Computer Interaction', 'Dr. Smith', '2025-03-27', '17:00:00', '18:30:00', 'A101', 115);

In [ ]:
select * from university_db.university_schema.lectures;

## Przykład: Wizualizacja danych w Pythonie

Teraz wykorzystamy wynik z poprzedniego zapytania SQL (zapisanego w `intro_sql`) i stworzymy interaktywną wizualizację za pomocą **Streamlit** i **Pandas**.

In [ ]:
# Importowanie niezbędnych bibliotek
import streamlit as st
import pandas as pd

# Możemy również użyć Snowpark do naszych analiz!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Następnie możemy użyć nazwy pythonowej, aby przekształcić intro_sql w Pandas dataframe
my_df = select_lectures.to_pandas()

# Grupowanie i zliczanie liczby wykładów w każdej sali
lecture_counts = my_df['ROOM_NUMBER'].value_counts().reset_index()
lecture_counts.columns = ['ROOM_NUMBER', 'count']

# Tworzenie wykresu danych
st.subheader("Liczba wykładów w salach 🎓")
st.bar_chart(lecture_counts, x='ROOM_NUMBER', y='count')

# Przegląd funkcji Snowflake

Snowflake to zaawansowana platforma do przechowywania i analizy danych w chmurze. Poniżej znajdziesz krótkie wprowadzenie do najważniejszych funkcji Snowflake.

---

## 1. Proste zapytanie SELECT

### Jak działają worksheety?

- **SQL Worksheets** to miejsce, gdzie możesz pisać i wykonywać zapytania SQL.

In [ ]:
SELECT 
    menu_item_name, 
    COUNT(*) AS order_count  -- Liczymy liczbę zamówień dla każdej pozycji menu
FROM 
    tb_101.analytics.orders_v
GROUP BY 
    menu_item_name
ORDER BY 
    order_count DESC  -- Sortujemy od najpopularniejszej pozycji
LIMIT 5;  -- Ograniczamy wynik do 5 pozycji

## 2. Warehouses

- **Warehouse** to zasoby obliczeniowe, które Snowflake używa do wykonywania zapytań.
- Możesz tworzyć różne warehouse'y dla różnych zadań, np.:
  - **Mały warehouse** do prostych zapytań.
  - **Duży warehouse** do złożonych analiz.


In [ ]:
CREATE WAREHOUSE my_warehouse WITH WAREHOUSE_SIZE = 'XSMALL'; // Stworzenie własnego warehouse z rozmiarem xsmall (mały)

## 3. Role

- **Role** w Snowflake określają, do jakich danych i funkcji użytkownik ma dostęp.
- **Role** w Snowflake to mechanizm, który określa, do jakich danych, obiektów i funkcji użytkownik ma dostęp. Każdy użytkownik w Snowflake ma przypisaną rolę (lub wiele ról), która definiuje jego uprawnienia. Role są kluczowym elementem zarządzania bezpieczeństwem i kontrolą dostępu w Snowflake.
- Role mogą dziedziczyć uprawnienia od innych ról, tworząc hierarchię.



## 4. Notebooks

- **Notebooks** to interaktywne środowisko do analizy danych, które łączy kod SQL, Python i Markdown.
- Możesz używać Notebooks do:
  - Eksploracji danych.
  - Tworzenia wizualizacji.
  - Dokumentowania analiz.

  ---


  ![notebooks](https://docs.snowflake.com/en/_images/sf-notebooks-intro.png)

# Snowflake Cortex: LLM in the cloud

[Snowflake Cortex](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions) to zestaw narzędzi, które umożliwiają dostęp do zaawansowanych modeli językowych (LLM) w środowisku Snowflake. Dzięki integracji z modelami takimi jak **Anthropic**, **Mistral**, **Reka**, **Meta**, **Gemini** oraz **Snowflake Arctic**, Cortex pozwala na wykorzystanie możliwości AI bez konieczności skomplikowanej konfiguracji. Snowflake Arctic to otwarty, korporacyjny model opracowany przez Snowflake, który oferuje dodatkowe możliwości analityczne.

---

## Na co zwrócić uwagę, korzystając ze Snowflake Cortex?

### **Bezpieczeństwo i zarządzanie danymi**  
- Modele są hostowane wewnątrz Snowflake, więc dane pozostają w kontrolowanym środowisku.  
- Nie ma potrzeby zarządzania infrastrukturą ML, ale jednocześnie dostępne są tylko predefiniowane modele.  

### **Integracja i łatwość wdrożenia**  
- Obsługa SQL i Python API pozwala na szybkie wdrożenie w istniejących procesach analitycznych.  
- Brak konieczności konfigurowania i trenowania modeli to oszczędność czasu, ale może ograniczać elastyczność w bardziej zaawansowanych przypadkach.  

### **Wydajność i koszty**  
- Snowflake Cortex pozwala na przetwarzanie dużych zbiorów danych wsadowo, co sprawdza się w analizie tekstu na dużą skalę.  
- REST API umożliwia interaktywne zapytania z niskimi opóźnieniami, np. w chatbotach.  
- Koszt przetwarzania może być wysoki przy intensywnym wykorzystaniu modeli NLP – warto monitorować zużycie zasobów.  

### **Ograniczenia ekosystemu**  
- Snowflake Cortex działa wyłącznie w ramach Snowflake, co oznacza brak możliwości użycia w innych środowiskach, np. on-premise.  
- Nie można trenować własnych modeli – dostępne są tylko te udostępnione przez Snowflake.  

---

## **Kiedy warto używać Snowflake Cortex?**

### **Typowe zastosowania:**
1. **Analiza dużych zbiorów danych**  
   - Wykorzystanie funkcji wsadowych do analizy sentymentu, klasyfikacji tekstu czy generowania podsumowań w SQL.  

2. **Interaktywne aplikacje**  
   - REST API pozwala na dynamiczne generowanie odpowiedzi w chatbotach lub systemach rekomendacyjnych.  

3. **Automatyzacja procesów biznesowych**  
   - Można wykorzystać Cortex do automatycznego tłumaczenia, klasyfikacji dokumentów czy ekstrakcji informacji.  

### **Kiedy warto rozważyć inne podejście?**
- Gdy potrzebujesz bardziej zaawansowanego dostosowania modeli NLP.  
- Jeśli chcesz optymalizować koszty i masz możliwość wdrożenia własnej infrastruktury ML.  
- W przypadku konieczności uruchamiania modeli poza Snowflake.  

Snowflake Cortex to wygodne rozwiązanie dla firm korzystających już ze Snowflake, ale warto rozważyć jego ograniczenia w kontekście konkretnych potrzeb biznesowych.

---

Koniec teorii, przejdźmy do przykładów. 🚀

# Przykład #0: Analiza sentymentu z użyciem Snowflake Cortex

---

Ten przykład pokazuje, jak użyć funkcji `SENTIMENT` z Snowflake Cortex do analizy sentymentu tekstu, podająć jako argument tekst.

In [ ]:
SELECT SNOWFLAKE.CORTEX.SENTIMENT('Snowflake is the best data platform that users can acess, it is so amazing that i can t stop thinking about it');


Ten przykład pokazuje, jak użyć funkcji `SENTIMENT` z Snowflake Cortex do analizy sentymentu tekstu, podająć jako argument tekst.

In [ ]:
select review_id, order_id, review, SNOWFLAKE.CORTEX.SENTIMENT(review) as sentiment_of_review, truck_brand_name from tb_voc.analytics.TRUCK_REVIEWS_V limit 10;

# Przykład #1: Zapytanie do modelu Mistral-Large

---

Ten przykład pokazuje, jak użyć funkcji `COMPLETE` z Snowflake Cortex, aby zadać pytanie modelowi językowemu **Mistral-Large**.

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'mistral-large2',
    'what is gen ai'
);

# Przykład #3: Podsumowanie tekstu

---

Ten przykład pokazuje, jak użyć funkcji `SUMMARIZE` z Snowflake Cortex do stworzenia skróconego podsumowania długiego tekstu.

In [ ]:
SELECT SNOWFLAKE.CORTEX.SUMMARIZE(
    'Artificial intelligence (AI) has rapidly evolved over the past few decades, transforming industries and revolutionizing the way humans interact with technology. Initially, AI systems were limited to rule-based programs that could only perform specific, predefined tasks. However, with advancements in machine learning and deep learning, modern AI has become increasingly capable of learning from vast amounts of data, recognizing patterns, and making autonomous decisions. These improvements have led to breakthroughs in natural language processing, computer vision, and robotics, allowing AI to be integrated into various applications, from virtual assistants like Siri and Alexa to self-driving cars and advanced medical diagnostics. Despite these advancements, ethical concerns and potential risks remain significant challenges, with debates surrounding issues like data privacy, bias in algorithms, and the potential impact of AI on the job market.

One of the most exciting aspects of AI is its potential to enhance human capabilities and solve complex global problems. For instance, AI is being used to predict climate change patterns, optimize supply chains, and even develop new drugs at unprecedented speeds. In healthcare, AI-powered systems assist doctors by analyzing medical images and suggesting accurate diagnoses, improving patient outcomes. However, as AI systems become more sophisticated, questions arise about their decision-making transparency and accountability. Many experts emphasize the need for robust regulatory frameworks and ethical guidelines to ensure AI is developed and deployed responsibly. While the future of AI holds great promise, it is crucial to strike a balance between innovation and ethical considerations to ensure that its benefits outweigh its risks.'
);

# Copilot: Asystent do analizy danych w Snowflake

[Copilot](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-agents) to narzędzie oparte na modelach językowych (LLM), które pomaga użytkownikom w analizie danych w środowisku Snowflake. Snowflake Copilot jest zintegrowany z platformą Snowflake i oferuje wsparcie w generowaniu zapytań SQL, eksploracji danych oraz optymalizacji istniejących rozwiązań. Działa w bezpiecznym środowisku Snowflake, co zapewnia kontrolę nad danymi i zgodność z zasadami RBAC (kontrola dostępu oparta na rolach).

---

## Czym jest Snowflake Copilot?

**Snowflake Copilot** to asystent, który:
- **Ułatwia analizę danych**: Pozwala na zadawanie pytań w języku naturalnym i generowanie odpowiedzi na podstawie danych.
- **Zapewnia bezpieczeństwo**: Działa w pełni wewnątrz Snowflake Cortex, co oznacza, że dane nigdy nie opuszczają bezpiecznego środowiska Snowflake.
- **Respektuje RBAC**: Dostarcza sugestie tylko na podstawie danych, do których użytkownik ma dostęp.

---

## Jak działa Snowflake Copilot?

1. **Eksploracja danych**:
   - Copilot pomaga zrozumieć strukturę danych. Można zapytać np.: *"Jakie kolumny znajdują się w tabeli X?"* lub *"Jakie są unikalne wartości w kolumnie Y?"*.
   - Asystent prowadzi użytkownika przez eksplorację nowych zbiorów danych, dostarczając przydatne informacje.

2. **Generowanie i optymalizacja zapytań SQL**:
   - Copilot generuje zapytania SQL na podstawie pytań w języku naturalnym, np.: *"Pokaż mi średnią sprzedaż w ostatnim kwartale."*
   - Może również sugerować optymalizacje istniejących zapytań lub poprawiać błędy.

3. **Pomoc w nauce SQL i Snowflake**:
   - Copilot dostarcza odpowiedzi na pytania dotyczące funkcjonalności Snowflake, np.: *"Jak stworzyć tabelę tymczasową?"*.
   - Odpowiedzi są oparte na oficjalnej dokumentacji Snowflake, co pomaga użytkownikom zwiększyć swoją biegłość w SQL.

---

## Gdzie można korzystać z Copilota?

Snowflake Copilot jest dostępny w:
- **SQL Worksheets**: Umożliwia generowanie i optymalizację zapytań SQL.
- **Snowflake Notebooks**: Integruje się z notebookami, aby wspierać analizę danych.

Aby skorzystać z Copilota, wystarczy otworzyć panel Copilota w Snowsight, wpisać pytanie, a Copilot dostarczy sugestie lub gotowe zapytania SQL.

---

## Przykłady użycia

1. **Eksploracja danych**:
   - *"Jakie kolumny znajdują się w tabeli `sales_data`?"*
   - *"Jakie są unikalne wartości w kolumnie `region`?"*

2. **Generowanie zapytań SQL**:
   - *"Pokaż mi top 10 klientów pod względem sprzedaży w 2023 roku."*
   - *"Jakie są średnie miesięczne przychody w każdym regionie?"*

3. **Optymalizacja zapytań**:
   - *"Czy to zapytanie można zoptymalizować?"*
   - *"Dlaczego to zapytanie działa wolno?"*

4. **Nauka Snowflake**:
   - *"Jak stworzyć nową tabelę w Snowflake?"*
   - *"Jak używać funkcji okna w SQL?"*

---

## O czym warto pamiętać?

Snowflake Copilot, choć potężny, ma swoje ograniczenia:
1. **Zaawansowane przypadki użycia**:
   - Copilot może nie radzić sobie z bardzo złożonymi zapytaniami SQL, wymagającymi specjalistycznej wiedzy.
   
2. **Koszty**:
   - Korzystanie z Copilota może wiązać się z dodatkowymi kosztami, szczególnie przy intensywnym wykorzystaniu.

3. **Zależność od jakości danych**:
   - Sugestie Copilota zależą od jakości i struktury danych. Nieprawidłowo zorganizowane dane mogą prowadzić do błędnych odpowiedzi.

---

## Podsumowanie

Snowflake Copilot to użyteczne narzędzie, które może znacznie ułatwić pracę z danymi, szczególnie dla użytkowników o mniejszym doświadczeniu w SQL. Jednak, jak każde rozwiązanie, ma swoje ograniczenia, które warto wziąć pod uwagę. Copilot najlepiej sprawdza się w prostszych zadaniach, takich jak eksploracja danych czy generowanie podstawowych zapytań SQL.


# Uruchomienie Copilota

W tym kroku uruchomimy **Snowflake Copilot**, który pomoże nam w analizie danych za pomocą prostych pytań w języku naturalnym. Copilot jest dostępny w **SQL Worksheets** i **Snowflake Notebooks**, a jego uruchomienie jest niezwykle proste.

---

## Krok 1: Stworzenie SQL Worksheet

1. **Przejdź do Snowsight**: Zaloguj się do Snowflake i otwórz **Snowsight**.
2. **Utwórz nowy SQL Worksheet**:
   - Kliknij na ikone **plusa** w lewym menu.
   - Następnie kliknij **SQL Worksheet**, aby utworzyć nowy arkusz.

![copilot_1](https://i.ibb.co/2YqMdCwk/copilot1.png)

---

## Krok 2: Uruchomienie widgeta Copilota

1. **Znajdź widget Copilota**:
   - W prawym dolnym rogu ekranu znajdziesz widget z napisem **Ask Copilot**.
   - Kliknij go, aby otworzyć panel Copilota.

![copilot_2](https://i.ibb.co/21sJdtjt/copilot2.png)

---

## Krok 3: Ustawienie bazy i schematu

1. **Wybierz bazę danych i schemat**:
   - Upewnij się, że w prawym górnym rogu SQL Worksheet wybrałeś odpowiednią **bazę danych** i **schemat**.
   - Copilot będzie korzystał z tych ustawień, aby generować zapytania SQL.

2. **Gotowe!**:
   - Teraz możesz korzystać z Copilota, aby zadawać pytania, generować zapytania SQL i optymalizować swoją pracę z danymi.


![copilot_3](https://speakaura-public.s3.eu-central-1.amazonaws.com/output.gif) 

# Cortex Search

W dzisiejszym świecie **dane** są jednym z najważniejszych filarów IT. Bez nich aplikacje, analizy i systemy decyzyjne nie miałyby sensu. Jednak ilość danych, które generujemy, jest ogromna.

W świecie **LLM (Large Language Models)** oraz **Snowflake** musimy radzić sobie z ogromnymi zbiorami danych. Aby uzyskać konkretne dane potrzebne do analizy za pomocą LLM, najpierw musimy przeszukać bazę danych (lub bazy danych), a następnie dostarczyć wszystkie pożądane fragmenty informacji. Co więcej, cały ten proces musi być przeprowadzony w sposób **optymalny**, aby zapewnić szybkość i efektywność.

---

## Czym jest Cortex Search?

**Cortex Search** to narzędzie Snowflake, które umożliwia **szybkie przeszukiwanie danych** (tzw. "[fuzzy search](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview)") z wykorzystaniem technik **hybrydowych** (łączenie wyszukiwania wektorowego i słów kluczowych). Dzięki Cortex Search możesz:

- **Przeszukiwać zbiory danych** w czasie rzeczywistym.
- **Automatycznie generować embeddingi** (wektory) dla tekstu, bez konieczności ręcznego ich tworzenia.
- **Zintegrować wyszukiwanie** z aplikacjami opartymi na LLM, takimi jak chatboty czy systemy rekomendacyjne.

Cortex Search jest szczególnie przydatny w dwóch głównych przypadkach użycia:

1. **Retrieval Augmented Generation (RAG)**:
   - Cortex Search działa jako **silnik RAG** dla chatbotów opartych na LLM.
   - Dzięki **wyszukiwaniu semantycznemu** dostarcza spersonalizowane i kontekstowe odpowiedzi, wykorzystując Twoje dane.

2. **Enterprise Search**:
   - Cortex Search może być używany jako **backend dla wyszukiwarki** w aplikacjach.
   - Umożliwia tworzenie **wysokiej jakości pasków wyszukiwania**, które są zintegrowane z Twoimi danymi w Snowflake.

![architecture](https://docs.snowflake.com/en/_images/cortex-search-rag.png)

---

## Zalety Costex Search?

- **Niska latencja**: Cortex Search zapewnia szybkie wyniki, nawet przy dużych zbiorach danych.
- **Brak konieczności zarządzania infrastrukturą**: Snowflake zajmuje się wszystkim – od generowania embeddingów po aktualizację indeksów.
- **Łatwość użycia**: Wystarczy kilka minut, aby skonfigurować wyszukiwanie na swoich danych.
- **Integracja z LLM**: Cortex Search idealnie współpracuje z modelami językowymi, dostarczając im dokładnie te dane, których potrzebują.

---

## Kiedy używać Cortex Search?

Cortex Search jest idealnym rozwiązaniem, gdy:

1. **Tworzysz chatboty oparte na LLM**:
   - Cortex Search dostarcza dane do modeli językowych, umożliwiając im generowanie **kontekstowych i spersonalizowanych odpowiedzi**.

2. **Potrzebujesz zaawansowanej wyszukiwarki w aplikacji**:
   - Cortex Search może być używany jako **backend dla wyszukiwarek** w aplikacjach korporacyjnych, zapewniając szybkie i dokładne wyniki.

---

## Przykłady użycia

1. **Chatbot dla klientów**:
   - Użytkownik pyta: "Jakie są najnowsze trendy w sprzedaży w Europie?"
   - Cortex Search dostarcza dane, a LLM generuje odpowiedź na podstawie aktualnych informacji.

2. **Wyszukiwarka dokumentów**:
   - Użytkownik wpisuje: "Znajdź wszystkie raporty z 2023 roku dotyczące COVID-19."
   - Cortex Search zwraca listę dokumentów, które pasują do zapytania.

---

## O czym warto pamiętać?

Cortex Search, choć potężny, ma swoje ograniczenia:

1. **Koszty**:
   - Korzystanie z Cortex Search może wiązać się z dodatkowymi kosztami, szczególnie przy intensywnym wykorzystaniu.

2. **Zależność od jakości danych**:
   - Efektywność wyszukiwania zależy od jakości i struktury danych. Nieprawidłowo zorganizowane dane mogą prowadzić do mniej precyzyjnych wyników.

3. **Krzywa uczenia się**:
   - Pomimo łatwości użycia, niektóre zaawansowane funkcje mogą wymagać czasu na opanowanie.

---
#### Dokumentacja Snowflake
- [Cortex Search Overview](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview)
- [Semantic Model Specification](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst/semantic-model-spec)

# Stworzenie Cortex Search Service

W tym kroku stworzymy **Cortex Search Service**, które następnie będziemy mogli łatwo zintegrować z innymi narzędziami Snowflake. Cortex Search Service umożliwia szybkie i efektywne przeszukiwanie danych, co jest kluczowe w pracy z dużymi zbiorami danych.

---

## Krok 0: Cortex Search Service

1. **Uruchom kreator Cortex Search Service**:
   - Przejdź do sekcji **AI & ML** w Snowflake.
   - Wybierz **Studio**, a następnie **Cortex Search Service**.

![search_1](https://i.ibb.co/NnSGyxwY/search1.png)

---

## Krok 1: Warehouse, baza i schemat

1. **Wybierz warehouse, bazę danych oraz schemat**:
   - Upewnij się, że wybierasz odpowiedni **warehouse**, **bazę danych** oraz **schemat**, w którym chcesz utworzyć serwis.

2. **Nadaj nazwę swojemu serwisowi**:
   - Wpisz unikalną nazwę dla swojego serwisu. Nazwa powinna być opisowa, aby łatwo było ją zidentyfikować w przyszłości.

![search_2](https://i.ibb.co/JWrT1W37/search2.png)

---

## Krok 2: Indeksowanie

1. **Wybór kolumny**: 
    - Wybierz kolumne, która ma być zidneksowana przez search service.
    - Po wybraniu kolumny kliknij przycisk na dole, by przejść dalej.

![search_3](https://i.ibb.co/wZfvqxmk/search3.png)

---

## Krok 3: Konfiguracja wyszukiwania

1. **Wybór kolumn**: 
    - Wybierz kolumny, które mają być brane pod uwagę podczas wyszukiwania
    - Po wybraniu kolumn kliknij przycisk na dole, by przejść dalej.

![search_4](https://i.ibb.co/Z1XQRJVX/search4.png)

---

Reszta kroków w kreatorze search service jest opcjonalna, na ten moment je pomijamy.

---

## Krok 4: Stwórz serwis

1. **Wszystko gotowe**: 
    - Upewnij się, że wszystkie potrzebne kolumny były zaznaczone, jeżeli tak kliknij przycisk na dole

![search_6](https://i.ibb.co/1fxkVjHC/search6.png)

---

To tyle i **aż tyle!**, wszystko w Snowflake robi się paroma kliknięciami 😊

# Cortex Analyst: Twój pomocnik w analizie danych

Cortex Analyst to narzędzie, które pozwala Ci skupić się na **analizie danych**, a nie na pisaniu skomplikowanych zapytań SQL.

- **Automatyczne przetwarzanie**: Tekst zostanie zamieniony na zapytanie SQL.
- **Wykonanie zapytania**: AI wykona zapytanie i zwróci wyniki.

**Analyst zdejmuje z Ciebie obowiązek pisania zaawansowanych instrukcji SQL.** 

- **Inteligentne podpowiedzi**: Jeśli AI nie zrozumie Twojego zapytania, zaproponuje, o co prawdopodobnie Ci chodziło.
- **Wyższa warstwa abstrakcji**: Wystarczy, że znasz dane oraz relacje między nimi. Zapytaj po ludzku, a Cortex Analyst zajmie się resztą.

Dzięki Cortex Analyst możesz:
- **Szybciej analizować dane** – bez konieczności pisania skomplikowanych zapytań.
- **Skupić się na wnioskach** – zamiast na technicznych szczegółach.
- **Pracować efektywniej** – nawet jeśli nie jesteś ekspertem od SQL.


# Przygotowanie danych (Tasty Bytes)

W tym kroku przygotujemy dane do analizy. Skupimy się na danych związanych z **Tasty Bytes**. Poniżej znajdziesz instrukcje krok po kroku, jak to zrobić.

---

## Krok 0: Cortex Analyst
- Uruchom kreator Cortex Analyst
- Przejdź do **AI & ML** -> **Studio** -> **Cortex Analyst**

![stage_0](https://i.ibb.co/XMZv06s/image-5.png)


## Krok 1: Stwórz semantic model

### 1. Wybierz stage
- Przejdź do sekcji **Stage** w Snowflake.

![stage_1](https://i.ibb.co/Xx1tVdN8/tb1.png)

- Wybierz odpowiedni stage, w którym znajdują się Twoje dane.

![stage_2](https://i.ibb.co/v6cZLCZn/tb2.png)

### 2. Zdefiniuj pytania
- Określ, jakie pytania chcesz zadać do danych.

![stage_3](https://i.ibb.co/F4NgCsWv/tb3.png)

### 3. Wybierz tabele i kolumny
- Wybierz tabele i kolumny, które będą potrzebne do odpowiedzi na Twoje pytania.
- Upewnij się, że wybrane kolumny zawierają odpowiednie dane.

![stage_4](https://i.ibb.co/C5WzTmHs/tb4.png)

### 4. Zapisz semantic model
- Po zdefiniowaniu pytań i wybraniu tabel, zapisz swój semantic model.
- Nadaj mu odpowiednią nazwę, np. `tasty_bytes`.

---

## Krok 2: Zacznij analizować za pomocą pytań

- Teraz, gdy masz gotowy semantic model i ustawione relacje, możesz zacząć analizować dane.
- Wprowadź swoje pytania w interfejsie Cortex Analyst i obserwuj, jak AI generuje odpowiedzi na podstawie Twoich danych.

![stage_final](https://speakaura-public.s3.eu-central-1.amazonaws.com/output1.gif)

---

**Gotowy?** No to do dzieła! 🚀 Cortex Analyst czeka, aby pomóc Ci w analizie danych. Powodzenia! 😊


# Cortex Agents: Inteligentne narzędzia AI w Snowflake

**[Cortex Agents](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-agents)** to jedna z najbardziej zaawansowanych funkcji AI dostępnych w Snowflake! Dzięki nim możemy stworzyć inteligentny interfejs, który pozwala na **seamless integration** (bezproblemową integrację) różnych funkcjonalności z platformy **Cortex**.

## Co potrafią Cortex Agents?

Agenci wykorzystują dwa potężne narzędzia:
1. **Cortex Search** – pozwala na efektywne wyszukiwanie i analizę danych.
2. **Cortex Analyst** – umożliwia zaawansowaną analizę semantyczną.

Dzięki połączeniu tych dwóch narzędzi, Cortex Agents stają się niezwykle potężnym narzędziem do pracy z danymi.

## Przykład użycia

W trakcie warsztatów pokażemy, jak **połączyć dwa modele semantyczne** za pomocą Cortex Agents. Następnie, wykorzystując prosty interfejs, przeanalizujemy **dwie bazy danych** – wszystko z jednego miejsca! To pokaże, jak łatwo i efektywnie można zarządzać danymi w Snowflake.

### Dlaczego to takie ważne?
- **Integracja**: Łączenie różnych funkcjonalności w jednym miejscu.
- **Efektywność**: Szybka analiza danych bez konieczności przełączania się między narzędziami.
- **Prostota**: Intuicyjny interfejs, który ułatwia pracę nawet początkującym użytkownikom.

Cortex Agents to przyszłość zarządzania danymi w Snowflake – przekonaj się sam!

# #1 Przygotowanie

W tej części przygotujemy środowisko do pracy z **Cortex Agents**. Wykonamy następujące kroki:

1. **Importowanie modeli i konfiguracja**:
   - Zaimportujemy niezbędne modele oraz przygotujemy pliki konfiguracyjne.
   - Wykorzystamy **Streamlit** do stworzenia prostego interfejsu graficznego, który umożliwi komunikację z agentem.

2. **Definiowanie ustawień API i modeli semantycznych**:
   - Skonfigurujemy endpoint API oraz modele semantyczne, które będą używane przez naszego agenta.
   - Wykorzystamy dwa modele semantyczne: jeden do analizy danych z bazy `TB_101`, a drugi do danych związanych z COVID-19.

Gotowy kod znajdziesz poniżej.

In [ ]:
# Importowanie niezbędnych bibliotek
import streamlit as st
import json
import _snowflake
from snowflake.snowpark.context import get_active_session

# Inicjalizacja sesji Snowflake
session = get_active_session()

# Konfiguracja endpointu API i timeoutu
API_ENDPOINT = "/api/v2/cortex/agent:run"
API_TIMEOUT = 50000  # timeout w milisekundach

# Definiowanie usług Cortex Search oraz modeli semantycznych
SEMANTIC_MODELS = "@tb_101.analytics.models/tasty.yaml"
COVID_SEMANTIC_MODELS = "@tb_101.analytics.models/covid.yaml"

# #2 Przygotowanie UI

W tym kroku zajmiemy się **stylizacją interfejsu użytkownika** za pomocą **CSS**. Naszym celem jest:
1. **Zdefiniowanie spójnej palety kolorów** dla całej aplikacji.
2. **Stylizacja komponentów** takich jak tytuły, pola tekstowe, przyciski, komunikaty o błędach i tooltipy.
3. **Ukrycie niepotrzebnych elementów** Streamlita, takich jak branding czy menu.

Poniżej znajdziesz gotowy kod CSS, który możesz dostosować do swoich potrzeb.

In [ ]:
st.markdown("""
<style>
/* Unified Color Palette */
:root {
    --background-color: #FFFFFF; /* Kolor tła aplikacji */
    --text-color: #222222;       /* Kolor tekstu */
    --title-color: #1A1A1A;      /* Kolor tytułów */
    --button-color: #1a56db;     /* Kolor przycisków */
    --button-text: #FFFFFF;      /* Kolor tekstu na przyciskach */
    --border-color: #CBD5E0;     /* Kolor obramowań */
    --accent-color: #2C5282;     /* Kolor akcentów */
}

/* General App Styling */
.stApp {
    background-color: var(--background-color); /* Tło aplikacji */
    color: var(--text-color);                  /* Kolor tekstu */
}

/* Title Styling */
h1, .stTitle {
    color: var(--title-color) !important;      /* Kolor tytułów */
    font-size: 36px !important;               /* Rozmiar czcionki */
    font-weight: 600 !important;              /* Grubość czcionki */
    padding: 1.5rem 0;                        /* Padding */
}

/* Input Fields */
textarea {
    background-color: white !important;        /* Tło pola tekstowego */
    border: 1px solid var(--border-color) !important; /* Obramowanie */
    border-radius: 4px !important;            /* Zaokrąglenie rogów */
    padding: 16px !important;                 /* Padding */
    font-size: 16px !important;               /* Rozmiar czcionki */
    color: var(--text-color) !important;      /* Kolor tekstu */
}

textarea:focus {
    border-color: var(--accent-color) !important; /* Kolor obramowania po kliknięciu */
    box-shadow: 0 0 0 1px var(--accent-color) !important; /* Cień */
}

textarea::placeholder {
    color: #666666 !important;                /* Kolor placeholderów */
}

/* Success & Error Messages */
.stException {
    background-color: #FEE2E2 !important;     /* Tło komunikatu o błędzie */
    border: 1px solid #EF4444 !important;     /* Obramowanie */
    padding: 16px !important;                 /* Padding */
    border-radius: 4px !important;            /* Zaokrąglenie rogów */
    margin: 16px 0 !important;                /* Margines */
    color: #991B1B !important;                /* Kolor tekstu */
}

div[data-testid="stAlert"], div[data-testid="stException"] {
    background-color: #f8d7da !important;     /* Tło alertu */
    color: #721c24 !important;                /* Kolor tekstu */
    border: 1px solid #f5c6cb !important;     /* Obramowanie */
    padding: 12px !important;                 /* Padding */
    border-radius: 6px !important;            /* Zaokrąglenie rogów */
    font-weight: bold !important;             /* Grubość czcionki */
}

div[data-testid="stAlertContentError"] {
    color: #721c24 !important;               /* Kolor tekstu błędu */
}

/* Buttons */
button[data-testid="stBaseButton-secondaryFormSubmit"] {
    background-color: #007bff !important;     /* Tło przycisku */
    color: white !important;                  /* Kolor tekstu */
    font-weight: bold !important;             /* Grubość czcionki */
    border-radius: 5px !important;            /* Zaokrąglenie rogów */
    padding: 8px 16px !important;             /* Padding */
    border: none !important;                  /* Brak obramowania */
}

/* Sidebar Buttons */
.stSidebar button {
    background-color: #1a56db !important;     /* Tło przycisku w sidebarze */
    color: white !important;                  /* Kolor tekstu */
    font-weight: 600 !important;              /* Grubość czcionki */
    border: none !important;                  /* Brak obramowania */
}

/* Tooltips */
.tooltip {
    visibility: hidden;                       /* Ukryj tooltip domyślnie */
    opacity: 0;                              /* Przezroczystość */
    background-color: white;                 /* Tło tooltipa */
    color: var(--text-color);                /* Kolor tekstu */
    padding: 10px;                           /* Padding */
    border-radius: 10px;                     /* Zaokrąglenie rogów */
    font-size: 14px;                         /* Rozmiar czcionki */
    line-height: 1.5;                        /* Wysokość linii */
    width: max-content;                      /* Szerokość */
    max-width: 300px;                        /* Maksymalna szerokość */
    position: absolute;                      /* Pozycja absolutna */
    z-index: 1000;                           /* Warstwa */
    bottom: calc(100% + 5px);               /* Pozycja względem elementu */
    left: 50%;                               /* Pozycja względem elementu */
    transform: translateX(-50%);             /* Centrowanie */
    transition: opacity 0.3s ease, transform 0.3s ease; /* Animacja */
}

.citation:hover + .tooltip {
    visibility: visible;                      /* Pokaż tooltip po najechaniu */
    opacity: 1;                              /* Pełna widoczność */
    transform: translateX(-50%) translateY(0); /* Animacja */
}

/* Hide Streamlit Branding */
#MainMenu, header, footer {
    visibility: hidden;                       /* Ukryj branding Streamlita */
}

/* Download Button */
[data-testid="stDownloadButton"] button {
    background-color: #2196F3 !important;     /* Tło przycisku pobierania */
    color: #FFFFFF !important;                /* Kolor tekstu */
    font-weight: 600 !important;              /* Grubość czcionki */
    border: none !important;                  /* Brak obramowania */
    padding: 0.5rem 1rem !important;          /* Padding */
    border-radius: 0.375rem !important;       /* Zaokrąglenie rogów */
    box-shadow: none !important;              /* Brak cienia */
}
</style>
""", unsafe_allow_html=True)

# #3 Ustawienie Agenta

W tym kroku skonfigurujemy naszego **Cortex Agenta**, aby był gotowy do działania. Wykonamy następujące czynności:

1. **Wybór modelu LLM**: Użyjemy zaawansowanego modelu językowego **Llama3.1-70b**, który będzie odpowiadał na zapytania użytkownika.
2. **Dodanie narzędzi**: Skonfigurujemy dostępne narzędzia, które agent będzie mógł wykorzystywać:
   - **Cortex Analyst Text-to-SQL**: Narzędzie do konwersji zapytań tekstowych na zapytania SQL.
   - Dwa modele semantyczne: `tasty1` (dla bazy `TB_101`) oraz `covid1` (dla bazy COVID-19).

Poniżej znajdziesz gotowy kod, który realizuje te zadania.

In [ ]:
def snowflake_api_call(query: str, limit: int = 10):
    # Przygotowanie payloadu dla API
    payload = {
        "model": "llama3.1-70b",  # Użyj modelu Llama3.1-70b
        "messages": [
            {
                "role": "user",  # Rola użytkownika
                "content": [
                    {
                        "type": "text",  # Typ wiadomości
                        "text": query    # Treść zapytania
                    }
                ]
            }
        ],
        "tools": [
            {
                "tool_spec": {
                    "type": "cortex_analyst_text_to_sql",  # Narzędzie do konwersji tekstu na SQL
                    "name": "tasty1"  # Nazwa narzędzia dla bazy TB_101
                }
            },
            {
                "tool_spec": {
                    "type": "cortex_analyst_text_to_sql",  # Narzędzie do konwersji tekstu na SQL
                    "name": "covid1"  # Nazwa narzędzia dla bazy COVID-19
                }
            }
        ],
        "tool_resources": {
            "tasty1": {"semantic_model_file": SEMANTIC_MODELS},  # Model semantyczny dla TB_101
            "covid1": {"semantic_model_file": COVID_SEMANTIC_MODELS}  # Model semantyczny dla COVID-19
        }
    }
    
    try:
        # Wysłanie zapytania do API Snowflake
        resp = _snowflake.send_snow_api_request(
            "POST",  # Metoda HTTP
            API_ENDPOINT,  # Endpoint API
            {},  # Nagłówki
            {},  # Parametry
            payload,  # Treść zapytania
            None,  # GUID zapytania
            API_TIMEOUT,  # Timeout w milisekundach
        )
        
        # Próba parsowania odpowiedzi JSON
        try:
            response_content = json.loads(resp["content"])
        except json.JSONDecodeError:
            st.error("❌ Nie udało się przetworzyć odpowiedzi API. Serwer mógł zwrócić nieprawidłowy format JSON.")
            if resp["status"] != 200:
                st.error(f"Błąd: {resp} ")
            return None
        return response_content
    except Exception as e:
        st.error(f"Błąd podczas wysyłania zapytania: {str(e)}")
        return None

# #4 Wykonanie zapytania SQL

W tym kroku stworzymy funkcję, która będzie odpowiedzialna za **wykonanie zapytania SQL** w Snowflake. Funkcja ta:
1. **Przyjmuje zapytanie SQL** jako argument.
2. **Usuwa średniki** z zapytania (jeśli istnieją), aby uniknąć błędów.
3. **Wykonuje zapytanie** za pomocą sesji Snowflake.
4. **Zwraca wynik** w postaci DataFrame lub wyświetla błąd, jeśli coś pójdzie nie tak.

Poniżej znajdziesz gotowy kod, który realizuje te zadania.

In [ ]:
def run_snowflake_query(query):
    """
    Wykonuje zapytanie SQL w Snowflake.

    Args:
        query (str): Zapytanie SQL do wykonania.

    Returns:
        DataFrame: Wynik zapytania w postaci DataFrame.
                  Zwraca None w przypadku błędu.
    """
    try:
        # Usuń średniki z zapytania, aby uniknąć błędów
        cleaned_query = query.replace(';', '')
        
        # Wykonaj zapytanie za pomocą sesji Snowflake
        df = session.sql(cleaned_query)
        return df
    except Exception as e:
        # Wyświetl błąd, jeśli coś pójdzie nie tak
        st.error(f"❌ Błąd podczas wykonywania zapytania SQL: {str(e)}")
        return None

# #5 Informacja zwrotna

W tym kroku zajmiemy się **przetwarzaniem odpowiedzi w formie streamingu** (SSE - Server-Sent Events). Odpowiedź od agenta przychodzi w postaci strumienia danych, dlatego musimy napisać funkcję, która:
1. **Parsuje dane** z odpowiedzi.
2. **Wydobywa tekst** oraz **zapytanie SQL** z otrzymanych danych.
3. **Obsługuje błędy**, aby zapewnić płynne działanie aplikacji.

Poniżej znajdziesz gotowy kod, który realizuje te zadania.

In [ ]:
def process_sse_response(response):
    """
    Przetwarza odpowiedź w formie streamingu (SSE) od agenta.

    Args:
        response (list): Lista zdarzeń SSE do przetworzenia.

    Returns:
        tuple: Krotka zawierająca tekst i zapytanie SQL.
               Jeśli odpowiedź jest pusta, zwraca puste stringi.
    """
    text = ""  # Przechowuje przetworzony tekst
    sql = ""   # Przechowuje przetworzone zapytanie SQL

    # Jeśli odpowiedź jest pusta, zwróć puste wartości
    if not response:
        return text, sql

    try:
        # Przetwarzaj każde zdarzenie w odpowiedzi
        for event in response:
            if event.get('event') == "message.delta":
                data = event.get('data', {})  # Dane z zdarzenia
                delta = data.get('delta', {})  # Delta zawierająca zmiany

                # Przetwarzaj każdy element zawartości
                for content_item in delta.get('content', []):
                    content_type = content_item.get('type')  # Typ zawartości

                    # Jeśli zawartość to wyniki narzędzi
                    if content_type == "tool_results":
                        tool_results = content_item.get('tool_results', {})
                        if 'content' in tool_results:
                            for result in tool_results['content']:
                                if result.get('type') == 'json':
                                    # Dodaj tekst z wyników JSON
                                    text += result.get('json', {}).get('text', '')
                                    # Przetwarzaj wyniki wyszukiwania
                                    search_results = result.get('json', {}).get('searchResults', [])
                                    for search_result in search_results:
                                        text += f"\n• {search_result.get('text', '')}"
                                    # Pobierz zapytanie SQL
                                    sql = result.get('json', {}).get('sql', '')

                    # Jeśli zawartość to zwykły tekst
                    if content_type == 'text':
                        text += content_item.get('text', '')  # Dodaj tekst

    except json.JSONDecodeError as e:
        st.error(f"❌ Błąd podczas parsowania zdarzeń: {str(e)}")
    except Exception as e:
        st.error(f"❌ Błąd podczas przetwarzania zdarzeń: {str(e)}")

    return text, sql

# #6 Funkcja main

W tym kroku zdefiniujemy **funkcję `main()`**, która jest sercem naszej aplikacji. Funkcja ta:
1. **Inicjalizuje interfejs użytkownika** za pomocą Streamlit.
2. **Obsługuje nowe konwersacje** poprzez przycisk w sidebarze.
3. **Przechowuje historię wiadomości** w stanie sesji.
4. **Wyświetla historię czatu** wraz z wygenerowanymi zapytaniami SQL i wynikami.
5. **Pobiera zapytania od użytkownika** i przetwarza je za pomocą wcześniej zdefiniowanych funkcji.

Poniżej znajdziesz gotowy kod, który realizuje te zadania.

In [ ]:
def main():
    """
    Główna funkcja aplikacji, która zarządza interfejsem użytkownika i logiką działania.
    """
    # Tytuł aplikacji
    st.title("Tasty bytes and COVID-19!")

    # Sidebar z przyciskiem do rozpoczynania nowej konwersacji
    with st.sidebar:
        if st.button("New Conversation", key="new_chat"):
            st.session_state.messages = []  # Resetowanie historii wiadomości
            st.rerun()  # Przeładowanie aplikacji

    # Inicjalizacja stanu sesji dla przechowywania wiadomości
    if 'messages' not in st.session_state:
        st.session_state.messages = []

    # Wyświetlanie historii wiadomości
    for message in st.session_state.messages:
        with st.chat_message(message['role']):  # Wyświetl wiadomość w odpowiednim stylu
            if message['role'] == 'assistant' and 'sql' in message:
                # Formatowanie i wyświetlanie odpowiedzi asystenta
                st.markdown(message['content'].replace("•", "\n\n-"))
                st.markdown("### Generated SQL")
                st.code(message['sql'], language="sql")  # Wyświetl wygenerowane SQL
                
                # Wykonaj zapytanie SQL i wyświetl wyniki
                sales_results = run_snowflake_query(message['sql'])
                if sales_results is not None:
                    st.write("### Result")
                    st.dataframe(sales_results)  # Wyświetl wyniki w postaci tabeli
            else:
                # Wyświetl zwykłą wiadomość
                st.markdown(message['content'].replace("•", "\n\n-"))

    # Pobierz zapytanie od użytkownika
    if query := st.chat_input("Start analyzing..."):
        # Dodaj wiadomość użytkownika do historii
        with st.chat_message("user"):
            st.markdown(query)
        st.session_state.messages.append({"role": "user", "content": query})
        
        # Przetwarzaj zapytanie za pomocą API Snowflake
        with st.spinner("Processing your request..."):
            response = snowflake_api_call(query, 1)  # Wywołanie API
            text, sql = process_sse_response(response)  # Przetwarzanie odpowiedzi
            
            # Dodaj odpowiedź asystenta do historii
            if text:
                st.session_state.messages.append({"role": "assistant", "content": text, "sql": sql})
                with st.chat_message("assistant"):
                    st.markdown(text.replace("•", "\n\n-"))  # Wyświetl odpowiedź asystenta
                    if sql:
                        st.markdown("### Generated SQL")
                        st.code(sql, language="sql")  # Wyświetl wygenerowane SQL
                        
                        # Wykonaj zapytanie SQL i wyświetl wyniki
                        sales_results = run_snowflake_query(sql)
                        if sales_results is not None:
                            st.write("### Result")
                            st.dataframe(sales_results)  # Wyświetl wyniki w postaci tabeli

# #7 Wykonanie

W tym kroku uruchomimy naszego agenta i zobaczymy, jak działa w praktyce.

Poniżej znajdziesz kod, który uruchamia naszą aplikację. Po prostu kliknij "Run" i ciesz się wynikami!

---

## Przykładowe prompty

- How many covid cases for each countries?
- List me menu item's names

In [ ]:
if __name__ == "__main__":
    # Uruchomienie aplikacji 
    main()